# WAPP Day-Ahead Market Clearing

**Objective**: Build a market clearing engine for the West African Power Pool (WAPP) day-ahead electricity market, progressively adding complexity.

| Step | Model | Key addition |
|------|-------|-------------|
| 1 | Single zone, single period | Basic welfare maximization |
| 2 | Multi-zone with NTC | Zonal pricing, congestion |
| 3 | Multi-period 24h | Hourly profiles, temporal dimension |
| 4 | Block orders | Fill-or-kill binary constraints |
| 5 | Complex orders | Linked orders, exclusive groups, MIC |


## Setup

In [1]:
import numpy as np
import pandas as pd
from pyomo.environ import *
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

---
## Step 1 — Single Zone, Single Period

### Problem

Given a set of supply offers $\mathcal{S}$ and demand bids $\mathcal{D}$, each with a price $p$ and quantity $q$, find the acceptance ratios that maximize **social welfare** (consumer surplus + producer surplus).

$$\max_{x^s, x^d} \quad \sum_{d \in \mathcal{D}} p_d \, q_d \, x^d_d \;-\; \sum_{s \in \mathcal{S}} p_s \, q_s \, x^s_s$$

**Subject to:**

$$\text{(Balance)} \quad \sum_{s \in \mathcal{S}} q_s \, x^s_s = \sum_{d \in \mathcal{D}} q_d \, x^d_d$$

$$0 \leq x^s_s \leq 1 \quad \forall s \in \mathcal{S}$$

$$0 \leq x^d_d \leq 1 \quad \forall d \in \mathcal{D}$$

The **clearing price** $\lambda$ is the dual variable of the balance constraint.

In [4]:
# --- Data ---
offers = [
    ("Solar",   800, 22), ("Gas_base", 1000, 28), ("Gas_mid", 600, 45),
    ("Gas_peak", 400, 72), ("Diesel",  200, 115),
]
demands = [
    ("City_A", 1200, 100), ("City_B", 800, 80),
    ("City_C", 500, 90),   ("City_D", 300, 60),
]

# --- Model ---
m = ConcreteModel()
S = range(len(offers));  D = range(len(demands))
m.xs = Var(S, bounds=(0,1))
m.xd = Var(D, bounds=(0,1))

m.obj = Objective(
    expr=sum(demands[d][2]*demands[d][1]*m.xd[d] for d in D)
        -sum(offers[s][2]*offers[s][1]*m.xs[s] for s in S),
    sense=maximize)

m.dual = Suffix(direction=Suffix.IMPORT)
m.bal = Constraint(
    expr=sum(offers[s][1]*m.xs[s] for s in S) == sum(demands[d][1]*m.xd[d] for d in D))

SolverFactory('glpk').solve(m, tee=False)

# --- Results ---
price = -m.dual[m.bal]
print(f"Welfare: {value(m.obj):,.0f} €")
print(f"Clearing price (λ): {price:.2f} €/MWh\n")

for s in S:
    v = value(m.xs[s])
    tag = "✓" if v > 0.99 else ("~" if v > 0.01 else "✗")
    print(f"  {tag} {offers[s][0]:12s}  {offers[s][1]:5d} MW @ {offers[s][2]:3d} €  →  {v*offers[s][1]:7.1f} MW accepted")
print()
for d in D:
    v = value(m.xd[d])
    tag = "✓" if v > 0.99 else ("~" if v > 0.01 else "✗")
    print(f"  {tag} {demands[d][0]:12s}  {demands[d][1]:5d} MW @ {demands[d][2]:3d} €  →  {v*demands[d][1]:7.1f} MW served")

Welfare: 149,200 €
Clearing price (λ): 72.00 €/MWh

  ✓ Solar           800 MW @  22 €  →    800.0 MW accepted
  ✓ Gas_base       1000 MW @  28 €  →   1000.0 MW accepted
  ✓ Gas_mid         600 MW @  45 €  →    600.0 MW accepted
  ~ Gas_peak        400 MW @  72 €  →    100.0 MW accepted
  ✗ Diesel          200 MW @ 115 €  →      0.0 MW accepted

  ✓ City_A         1200 MW @ 100 €  →   1200.0 MW served
  ✓ City_B          800 MW @  80 €  →    800.0 MW served
  ✓ City_C          500 MW @  90 €  →    500.0 MW served
  ✗ City_D          300 MW @  60 €  →      0.0 MW served


In [5]:
# Merit order visualization
fig = make_subplots(rows=1, cols=1)
so = sorted(range(len(offers)), key=lambda i: offers[i][2])
cum = 0
for i in so:
    name, q, p = offers[i]
    color = "rgba(39,174,96,0.7)" if p <= price else "rgba(231,76,60,0.5)"
    fig.add_trace(go.Bar(x=[cum + q/2], y=[p], width=[q], marker_color=color,
        name=name, showlegend=True, hovertemplate=f"<b>{name}</b><br>{q} MW @ {p} €/MWh"))
    cum += q

sd = sorted(range(len(demands)), key=lambda i: -demands[i][2])
cum = 0
xs, ys = [0], [demands[sd[0]][2]]
for i in sd:
    _, q, p = demands[i]
    xs += [cum, cum+q]; ys += [p, p]; cum += q
fig.add_trace(go.Scatter(x=xs, y=ys, mode='lines', name='Demand curve',
    line=dict(color='rgba(52,152,219,0.9)', width=2.5)))

fig.add_hline(y=price, line_dash="dash", line_color="orange",
    annotation_text=f"λ = {price:.1f} €/MWh")
fig.update_layout(title="Step 1 — Merit Order & Clearing Price",
    xaxis_title="Cumulative MW", yaxis_title="€/MWh", barmode='overlay', height=420)
fig.show()

---
## Step 2 — Multi-Zone with NTC Constraints

### Problem

Now the market spans $|\mathcal{Z}|$ zones connected by transmission lines with Net Transfer Capacity (NTC) limits. Each line $(u,v)$ allows flow in one direction at a time.

$$\max \quad \sum_{d \in \mathcal{D}} p_d \, q_d \, x^d_d \;-\; \sum_{s \in \mathcal{S}} p_s \, q_s \, x^s_s$$

**Subject to:**

$$\text{(Nodal balance)} \quad \forall z \in \mathcal{Z}: \quad \sum_{s \in \mathcal{S}_z} q_s x^s_s + \text{imports}_z - \text{exports}_z = \sum_{d \in \mathcal{D}_z} q_d x^d_d$$

where:
$$\text{imports}_z = \sum_{(u,z) \in \mathcal{L}} f_{u,z} + \sum_{(z,v) \in \mathcal{L}} f^r_{z,v}$$

$$\text{(NTC)} \quad f_{u,v} \leq \text{NTC}_{u,v} \cdot b_{u,v}, \quad f^r_{u,v} \leq \text{NTC}_{u,v} \cdot (1 - b_{u,v})$$

$$b_{u,v} \in \{0,1\} \quad \text{(flow direction selector)}$$

The **zonal price** $\lambda_z$ is the dual of the balance constraint for zone $z$.  
When an NTC is saturated, prices diverge → **market splitting**.

In [6]:
# --- WAPP Network: 14 zones, 15 interconnections ---
ZONES = ['NGA','BEN','TGO','GHA','CIV','BFA','MLI','SEN','GIN','SLE','LBR','GNB','GMB','NER']

LINES = [
    ('NGA','BEN',800), ('NGA','NER',300), ('BEN','TGO',600), ('TGO','GHA',500),
    ('GHA','CIV',600), ('GHA','BFA',250), ('CIV','BFA',250), ('CIV','MLI',250),
    ('CIV','LBR',400), ('LBR','SLE',400), ('SLE','GIN',400), ('GIN','GNB',300),
    ('GNB','GMB',300), ('GMB','SEN',300), ('SEN','MLI',300),
]
NTC = {(u,v): c for u,v,c in LINES}
PAIRS = list(NTC.keys())

# Offers: (name, zone, MW, €/MWh)
offers = [
    ("Mainstream Solar",'NGA',800,22), ("Egbin Gas",'NGA',1000,28),
    ("Delta Gas",'NGA',600,30),        ("Geregu",'NGA',400,32),
    ("Afam VI",'NGA',500,38),          ("Olorunsogo",'NGA',600,40),
    ("VRA Akosombo",'GHA',900,30),     ("Sunon Asogli",'GHA',300,52),
    ("Cenpower",'GHA',200,62),         ("Karpowership",'GHA',400,72),
    ("CI-Energies Hydro",'CIV',600,28),("CIPREL Gaz",'CIV',400,45),
    ("Azito",'CIV',300,50),            ("Aggreko CIV",'CIV',100,92),
    ("OMVS Manantali",'SEN',150,38),   ("OMVS Felou",'MLI',150,38),
    ("SENELEC Thermal",'SEN',400,115), ("OMVG Kaleta",'GIN',100,42),
    ("EDG Garafiri",'GIN',100,82),     ("ContourGlobal",'TGO',100,98),
    ("CEB Nangbeto",'BEN',50,105),     ("SONABEL",'BFA',150,142),
    ("EDM-SA",'MLI',200,138),          ("NAWEC",'GMB',50,155),
    ("EAGB",'GNB',30,165),             ("LEC",'LBR',80,148),
    ("EDSA",'SLE',60,155),             ("NIGELEC Gen",'NER',80,132),
]

demands = [
    ("TCN Nigeria",'NGA',3500,120),  ("ECG Ghana",'GHA',1800,130),
    ("NEDCO",'GHA',400,125),         ("CIE Distrib.",'CIV',1600,150),
    ("SBEE",'BEN',400,160),          ("CEET",'TGO',300,155),
    ("SENELEC",'SEN',700,170),       ("SONABEL",'BFA',500,200),
    ("EDM-SA",'MLI',550,190),        ("NIGELEC",'NER',350,210),
    ("EDG Guinee",'GIN',400,160),    ("EDSA",'SLE',150,180),
    ("LEC",'LBR',120,175),           ("NAWEC",'GMB',80,190),
    ("EAGB",'GNB',50,195),
]

S = range(len(offers)); D = range(len(demands))

# --- Model ---
m = ConcreteModel()
m.xs = Var(S, bounds=(0,1))
m.xd = Var(D, bounds=(0,1))
m.f  = Var(PAIRS, domain=NonNegativeReals)   # forward flow
m.fr = Var(PAIRS, domain=NonNegativeReals)   # reverse flow
m.b  = Var(PAIRS, domain=Binary)             # direction selector

m.obj = Objective(
    expr=sum(demands[d][3]*demands[d][2]*m.xd[d] for d in D)
        -sum(offers[s][3]*offers[s][2]*m.xs[s] for s in S),
    sense=maximize)

m.dual = Suffix(direction=Suffix.IMPORT)

# Nodal balance for each zone
def balance(m, z):
    prod = sum(offers[s][2]*m.xs[s] for s in S if offers[s][1]==z)
    cons = sum(demands[d][2]*m.xd[d] for d in D if demands[d][1]==z)
    imp  = sum(m.f[u,v] for (u,v) in PAIRS if v==z) + sum(m.fr[u,v] for (u,v) in PAIRS if u==z)
    exp  = sum(m.f[u,v] for (u,v) in PAIRS if u==z) + sum(m.fr[u,v] for (u,v) in PAIRS if v==z)
    return prod + imp - exp == cons
m.bal = Constraint(ZONES, rule=balance)

# NTC limits
m.ntc_fwd = Constraint(PAIRS, rule=lambda m,u,v: m.f[u,v] <= NTC[(u,v)] * m.b[u,v])
m.ntc_rev = Constraint(PAIRS, rule=lambda m,u,v: m.fr[u,v] <= NTC[(u,v)] * (1 - m.b[u,v]))

# Solve MIP then fix binaries for LP duals
SolverFactory('glpk').solve(m, tee=False)
for (u,v) in PAIRS:
    m.b[u,v].fix(round(value(m.b[u,v])))
SolverFactory('glpk').solve(m, tee=False)

# Zonal prices from duals
prices = {}
for z in ZONES:
    p = -m.dual.get(m.bal[z], 0)
    if p < 0:  # fallback: marginal accepted offer
        acc = [(offers[s][3], value(m.xs[s])) for s in S if offers[s][1]==z and value(m.xs[s])>0.01]
        p = max([pr for pr,_ in acc], default=0)
    prices[z] = round(p, 2)

print(f"Social welfare: {value(m.obj):,.0f} €\n")
print("Zonal prices:")
for z in sorted(ZONES, key=lambda z: prices[z]):
    print(f"  {z}: {prices[z]:7.2f} €/MWh")

Social welfare: 874,910 €

Zonal prices:
  GHA:   72.00 €/MWh
  NGA:  120.00 €/MWh
  BEN:  130.00 €/MWh
  TGO:  130.00 €/MWh
  NER:  132.00 €/MWh
  CIV:  150.00 €/MWh
  BFA:  150.00 €/MWh
  MLI:  160.00 €/MWh
  SEN:  160.00 €/MWh
  GIN:  160.00 €/MWh
  SLE:  160.00 €/MWh
  LBR:  160.00 €/MWh
  GNB:  160.00 €/MWh
  GMB:  160.00 €/MWh


In [7]:
# Flows
flows = []
for (u,v) in PAIRS:
    fwd, rev = value(m.f[u,v]), value(m.fr[u,v])
    if fwd > 0.1:
        flows.append((u, v, fwd, fwd/NTC[(u,v)]*100))
    if rev > 0.1:
        flows.append((v, u, rev, rev/NTC[(u,v)]*100))

print("Cross-border flows:")
for u, v, mw, pct in sorted(flows, key=lambda x: -x[2]):
    tag = "🔴" if pct > 95 else ("🟡" if pct > 50 else "🟢")
    print(f"  {tag} {u} → {v}: {mw:7.1f} MW  ({pct:.0f}% of NTC)")

# Net positions
positions = {}
for z in ZONES:
    prod = sum(offers[s][2]*value(m.xs[s]) for s in S if offers[s][1]==z)
    cons = sum(demands[d][2]*value(m.xd[d]) for d in D if demands[d][1]==z)
    positions[z] = round(prod - cons, 1)

print("\nNet positions (>0 = exporter):")
for z in sorted(ZONES, key=lambda z: -positions[z]):
    arrow = "→ export" if positions[z] > 0 else "← import"
    print(f"  {z}: {positions[z]:+8.1f} MW  {arrow}")

Cross-border flows:
  🔴 NGA → BEN:   800.0 MW  (100% of NTC)
  🔴 GHA → CIV:   600.0 MW  (100% of NTC)
  🟡 BEN → TGO:   450.0 MW  (75% of NTC)
  🔴 CIV → LBR:   400.0 MW  (100% of NTC)
  🟡 LBR → SLE:   360.0 MW  (90% of NTC)
  🔴 NGA → NER:   300.0 MW  (100% of NTC)
  🟡 SLE → GIN:   270.0 MW  (68% of NTC)
  🟢 TGO → GHA:   250.0 MW  (50% of NTC)
  🔴 GHA → BFA:   250.0 MW  (100% of NTC)
  🔴 CIV → MLI:   250.0 MW  (100% of NTC)
  🟡 GIN → GNB:   180.0 MW  (60% of NTC)
  🟢 GNB → GMB:   130.0 MW  (43% of NTC)
  🟢 CIV → BFA:   100.0 MW  (40% of NTC)
  🟢 GMB → SEN:   100.0 MW  (33% of NTC)
  🟢 MLI → SEN:    50.0 MW  (17% of NTC)

Net positions (>0 = exporter):
  NGA:  +1100.0 MW  → export
  GHA:   +600.0 MW  → export
  CIV:   +150.0 MW  → export
  GMB:    -30.0 MW  ← import
  LBR:    -40.0 MW  ← import
  GNB:    -50.0 MW  ← import
  GIN:    -90.0 MW  ← import
  SLE:    -90.0 MW  ← import
  SEN:   -150.0 MW  ← import
  TGO:   -200.0 MW  ← import
  MLI:   -200.0 MW  ← import
  NER:   -300.0 MW  ← i

In [8]:
# Price map visualization
ZONE_NAMES = {
    'NGA':'Nigeria','BEN':'Benin','TGO':'Togo','GHA':'Ghana','CIV':'Côte d\'Ivoire',
    'BFA':'Burkina Faso','MLI':'Mali','SEN':'Senegal','GIN':'Guinea','SLE':'Sierra Leone',
    'LBR':'Liberia','GNB':'Guinea-Bissau','GMB':'Gambia','NER':'Niger'
}

df = pd.DataFrame([
    {'Zone': z, 'Country': ZONE_NAMES[z], 'Price': prices[z], 'Position': positions[z]}
    for z in ZONES
]).sort_values('Price')

fig = make_subplots(rows=1, cols=2, column_widths=[0.6, 0.4],
    subplot_titles=["Zonal Prices", "Net Positions"])

avg = df['Price'].mean()
fig.add_trace(go.Bar(
    y=df['Country'], x=df['Price'], orientation='h',
    marker_color=[f"rgba(39,174,96,0.8)" if p <= avg else "rgba(231,76,60,0.8)" for p in df['Price']],
    text=[f"{p:.0f}" for p in df['Price']], textposition='outside',
    showlegend=False), row=1, col=1)
fig.add_vline(x=avg, line_dash="dash", line_color="orange", row=1, col=1)

colors = ["rgba(39,174,96,0.8)" if p > 0 else "rgba(52,152,219,0.8)" for p in df['Position']]
fig.add_trace(go.Bar(
    y=df['Country'], x=df['Position'], orientation='h', marker_color=colors,
    showlegend=False), row=1, col=2)
fig.add_vline(x=0, line_color="gray", row=1, col=2)

fig.update_layout(height=500, title="Step 2 — Multi-Zone Clearing Results")
fig.update_xaxes(title_text="€/MWh", row=1, col=1)
fig.update_xaxes(title_text="MW (>0 = export)", row=1, col=2)
fig.show()

---
## Step 3 — Multi-Period 24h

### Problem

Extend to $T = 24$ hourly periods. Each offer $s$ now has a **generation profile** $\bar{q}_{s,t}$ (capacity available at hour $t$), and each demand $d$ has a **load profile** $\bar{q}_{d,t}$.

$$\max \quad \sum_{t \in \mathcal{T}} \left[ \sum_{d \in \mathcal{D}} p_d \, \bar{q}_{d,t} \, x^d_{d,t} \;-\; \sum_{s \in \mathcal{S}} p_s \, \bar{q}_{s,t} \, x^s_{s,t} \right]$$

**Subject to** $\forall z \in \mathcal{Z}, \; \forall t \in \mathcal{T}$:

$$\sum_{s \in \mathcal{S}_z} \bar{q}_{s,t} \, x^s_{s,t} + \text{imports}_{z,t} - \text{exports}_{z,t} = \sum_{d \in \mathcal{D}_z} \bar{q}_{d,t} \, x^d_{d,t}$$

$$f_{u,v,t} \leq \text{NTC}_{u,v} \cdot b_{u,v,t}, \quad f^r_{u,v,t} \leq \text{NTC}_{u,v} \cdot (1 - b_{u,v,t})$$

$$0 \leq x^s_{s,t}, x^d_{d,t} \leq 1, \quad b_{u,v,t} \in \{0,1\}$$

The **zonal price** at hour $t$ is $\lambda_{z,t}$ = dual of balance constraint $(z,t)$.

### Generation profiles

| Type | Shape | Example |
|------|-------|---------|
| Solar | Bell curve 06h–18h | Mainstream Solar (NGA) |
| Hydro | Modulated 60–100% | VRA Akosombo (GHA) |
| Baseload | Flat ~95% | Egbin Gas (NGA) |
| Peaker | 100% available | SENELEC Thermal (SEN) |

### Load profile (West Africa typical)
Trough at 04h (~47% of peak), morning rise, evening peak at 19h (100%).

In [9]:
# --- Hourly profiles (normalized 0-1) ---
T = list(range(24))

PROF = {
    'baseload': [0.95]*24,
    'hydro': [0.7,0.65,0.6,0.6,0.6,0.65,0.75,0.85,0.9,0.95,1.0,1.0,
              1.0,0.95,0.9,0.85,0.9,0.95,1.0,0.95,0.9,0.85,0.8,0.75],
    'solar':  [0,0,0,0,0,0.05,0.2,0.45,0.7,0.85,0.95,1.0,
               1.0,0.95,0.85,0.7,0.45,0.2,0.05,0,0,0,0,0],
    'peaker': [1.0]*24,
}
LOAD_WA = [0.55,0.50,0.48,0.47,0.48,0.52,0.62,0.75,0.85,0.90,0.92,0.90,
           0.88,0.85,0.82,0.80,0.82,0.88,0.95,1.00,0.98,0.90,0.78,0.65]

# Supply: (name, zone, Pmax, price, profile_type)
supply = [
    ("Mainstream Solar",'NGA',800,22,'solar'), ("Egbin Gas",'NGA',1000,28,'baseload'),
    ("Delta Gas",'NGA',600,30,'baseload'),     ("Geregu",'NGA',400,32,'baseload'),
    ("Afam VI",'NGA',500,38,'baseload'),       ("Olorunsogo",'NGA',600,40,'peaker'),
    ("VRA Akosombo",'GHA',900,30,'hydro'),     ("Sunon Asogli",'GHA',300,52,'baseload'),
    ("Cenpower",'GHA',200,62,'baseload'),      ("Karpowership",'GHA',400,72,'peaker'),
    ("CI-Energies",'CIV',600,28,'hydro'),      ("CIPREL",'CIV',400,45,'baseload'),
    ("Azito",'CIV',300,50,'baseload'),         ("Aggreko",'CIV',100,92,'peaker'),
    ("OMVS Manantali",'SEN',150,38,'hydro'),   ("OMVS Felou",'MLI',150,38,'hydro'),
    ("SENELEC Therm.",'SEN',400,115,'peaker'), ("OMVG Kaleta",'GIN',100,42,'hydro'),
    ("EDG Garafiri",'GIN',100,82,'hydro'),     ("ContourGlobal",'TGO',100,98,'baseload'),
    ("CEB Nangbeto",'BEN',50,105,'hydro'),     ("SONABEL",'BFA',150,142,'peaker'),
    ("EDM-SA",'MLI',200,138,'peaker'),         ("NAWEC",'GMB',50,155,'peaker'),
    ("EAGB",'GNB',30,165,'peaker'),            ("LEC",'LBR',80,148,'peaker'),
    ("EDSA",'SLE',60,155,'peaker'),            ("NIGELEC",'NER',80,132,'peaker'),
]

# Demand: (name, zone, peak_MW, max_price)
demand = [
    ("TCN Nigeria",'NGA',3500,120),  ("ECG Ghana",'GHA',1800,130),
    ("NEDCO",'GHA',400,125),         ("CIE",'CIV',1600,150),
    ("SBEE",'BEN',400,160),          ("CEET",'TGO',300,155),
    ("SENELEC",'SEN',700,170),       ("SONABEL",'BFA',500,200),
    ("EDM-SA",'MLI',550,190),        ("NIGELEC",'NER',350,210),
    ("EDG",'GIN',400,160),           ("EDSA",'SLE',150,180),
    ("LEC",'LBR',120,175),           ("NAWEC",'GMB',80,190),
    ("EAGB",'GNB',50,195),
]

# Build hourly capacity/load arrays
Qs = {s: [round(supply[s][2] * PROF[supply[s][4]][t]) for t in T] for s in range(len(supply))}
Qd = {d: [round(demand[d][2] * LOAD_WA[t]) for t in T] for d in range(len(demand))}

S = range(len(supply)); D = range(len(demand))

print(f"Supply: {len(supply)} offers × 24h = {sum(sum(Qs[s]) for s in S):,} MWh available")
print(f"Demand: {len(demand)} bids   × 24h = {sum(sum(Qd[d]) for d in D):,} MWh requested")

Supply: 28 offers × 24h = 186,062 MWh available
Demand: 15 bids   × 24h = 198,925 MWh requested


In [10]:
# --- 24h Multi-Zone Model ---
import time
t0 = time.time()

m = ConcreteModel()
m.xs = Var(S, T, bounds=(0,1))
m.xd = Var(D, T, bounds=(0,1))
m.f  = Var(PAIRS, T, domain=NonNegativeReals)
m.fr = Var(PAIRS, T, domain=NonNegativeReals)
m.b  = Var(PAIRS, T, domain=Binary)

# Objective: total welfare over 24h
m.obj = Objective(expr=
    sum(demand[d][3]*Qd[d][t]*m.xd[d,t] for t in T for d in D if Qd[d][t]>0)
  - sum(supply[s][3]*Qs[s][t]*m.xs[s,t] for t in T for s in S if Qs[s][t]>0),
    sense=maximize)

m.dual = Suffix(direction=Suffix.IMPORT)

# Nodal balance per zone per hour
def balance(m, z, t):
    prod = sum(Qs[s][t]*m.xs[s,t] for s in S if supply[s][1]==z and Qs[s][t]>0)
    cons = sum(Qd[d][t]*m.xd[d,t] for d in D if demand[d][1]==z and Qd[d][t]>0)
    imp  = sum(m.f[u,v,t] for (u,v) in PAIRS if v==z) + sum(m.fr[u,v,t] for (u,v) in PAIRS if u==z)
    exp  = sum(m.f[u,v,t] for (u,v) in PAIRS if u==z) + sum(m.fr[u,v,t] for (u,v) in PAIRS if v==z)
    return prod + imp - exp == cons
m.bal = Constraint(ZONES, T, rule=balance)

m.ntc_f = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.f[u,v,t] <= NTC[(u,v)]*m.b[u,v,t])
m.ntc_r = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.fr[u,v,t] <= NTC[(u,v)]*(1-m.b[u,v,t]))

# Solve
SolverFactory('glpk').solve(m, tee=False)
for (u,v) in PAIRS:
    for t in T:
        m.b[u,v,t].fix(round(value(m.b[u,v,t])))
SolverFactory('glpk').solve(m, tee=False)

dt = time.time() - t0
print(f"Solved in {dt:.1f}s")
print(f"Total welfare (24h): {value(m.obj):,.0f} €")

# Extract zonal prices
prices_zt = {}
for z in ZONES:
    prices_zt[z] = []
    for t in T:
        p = -m.dual.get(m.bal[z,t], 0)
        if p < 0:
            acc = [(supply[s][3], value(m.xs[s,t])) for s in S
                   if supply[s][1]==z and Qs[s][t]>0
                   and value(m.xs[s,t]) is not None and value(m.xs[s,t])>0.01]
            p = max([pr for pr,_ in acc], default=0)
        prices_zt[z].append(round(p, 2))

Solved in 2.1s
Total welfare (24h): 17,450,604 €


In [11]:
# 24h price curves
fig = go.Figure()
for z in ZONES:
    fig.add_trace(go.Scatter(x=T, y=prices_zt[z], mode='lines', name=z,
        hovertemplate=f'{ZONE_NAMES[z]}<br>H%{{x}}: %{{y:.1f}} €/MWh'))
fig.update_layout(title="Step 3 — Zonal Prices over 24h",
    xaxis_title="Hour", yaxis_title="€/MWh", height=450,
    legend=dict(orientation='h', y=-0.15))
fig.update_xaxes(dtick=2)
fig.show()

In [12]:
# Price heatmap
fig = go.Figure(go.Heatmap(
    z=[[prices_zt[z][t] for z in ZONES] for t in T],
    x=[f"{z}" for z in ZONES],
    y=[f"H{t:02d}" for t in T],
    colorscale='RdYlBu_r',
    colorbar=dict(title='€/MWh'),
    hovertemplate='%{x} — %{y}<br>%{z:.1f} €/MWh<extra></extra>'))
fig.update_layout(title="Price Heatmap (Zone × Hour)", height=520)
fig.show()

In [13]:
# Dispatch stack by technology
type_colors = {'solar':'#F1C40F','hydro':'#3498DB','baseload':'#2ECC71','peaker':'#E74C3C'}
fig = go.Figure()
for tp in ['solar','hydro','baseload','peaker']:
    hourly = [0]*24
    for s in S:
        if supply[s][4] == tp:
            for t in T:
                if Qs[s][t] > 0:
                    hourly[t] += Qs[s][t] * (value(m.xs[s,t]) if Qs[s][t]>0 else 0)
    fig.add_trace(go.Scatter(x=T, y=hourly, mode='lines', name=tp.capitalize(),
        stackgroup='one', line=dict(width=0.5)))

# Demand served
dem_h = [0]*24
for d in D:
    for t in T:
        if Qd[d][t] > 0:
            dem_h[t] += Qd[d][t] * value(m.xd[d,t])
fig.add_trace(go.Scatter(x=T, y=dem_h, mode='lines', name='Demand served',
    line=dict(color='black', width=2, dash='dash')))

fig.update_layout(title="Generation Dispatch by Technology — 24h",
    xaxis_title="Hour", yaxis_title="MW", height=450)
fig.update_xaxes(dtick=2)
fig.show()

In [14]:
# Welfare decomposition per hour
welfare_h = []
for t in T:
    w = sum(demand[d][3]*Qd[d][t]*value(m.xd[d,t]) for d in D if Qd[d][t]>0) \
      - sum(supply[s][3]*Qs[s][t]*value(m.xs[s,t]) for s in S if Qs[s][t]>0)
    welfare_h.append(round(w))

fig = go.Figure(go.Bar(x=T, y=welfare_h, marker_color='rgba(39,174,96,0.7)'))
fig.update_layout(title="Hourly Social Welfare", xaxis_title="Hour", yaxis_title="€", height=380)
fig.update_xaxes(dtick=2)
fig.show()

print(f"Sum of hourly welfare: {sum(welfare_h):,.0f} €")
print(f"Objective value:       {value(m.obj):,.0f} €")

Sum of hourly welfare: 17,450,604 €
Objective value:       17,450,604 €


---
## Step 4 — Block Orders

### Problem

A **block order** $b$ is an all-or-nothing bid spanning a set of consecutive hours $\mathcal{T}_b$. It has a single price $p_b$ and a fixed power $q_b$. It must be either **fully accepted or fully rejected** across all hours in its block.

Introduce binary variable $y_b \in \{0,1\}$:

$$\max \quad \underbrace{\sum_{t} \sum_{d} p_d \bar{q}_{d,t} x^d_{d,t} - \sum_{t} \sum_{s} p_s \bar{q}_{s,t} x^s_{s,t}}_{\text{hourly orders}}
\;+\; \underbrace{\sum_{b \in \mathcal{B}^D} p_b q_b |\mathcal{T}_b| \, y_b - \sum_{b \in \mathcal{B}^S} p_b q_b |\mathcal{T}_b| \, y_b}_{\text{block orders}}$$

**Additional constraints** (block injection/withdrawal in balance):

$$\forall z, t: \quad \text{balance includes } \sum_{b \in \mathcal{B}^S_z: t \in \mathcal{T}_b} q_b \, y_b \text{ (supply blocks)}$$

$$y_b \in \{0,1\} \quad \forall b \in \mathcal{B}$$

### Acceptance condition (Paradoxically Accepted/Rejected)

A block is **in the money** if its average clearing price over its hours covers its bid price:

$$\text{Supply block}: \quad \frac{1}{|\mathcal{T}_b|} \sum_{t \in \mathcal{T}_b} \lambda_{z,t} \geq p_b$$

$$\text{Demand block}: \quad \frac{1}{|\mathcal{T}_b|} \sum_{t \in \mathcal{T}_b} \lambda_{z,t} \leq p_b$$

A **Paradoxically Rejected Block (PRB)** is in-the-money but rejected. A **Paradoxically Accepted Block (PAB)** is out-of-the-money but accepted. EUPHEMIA minimizes these.

In [19]:
# Reuse data from Step 3, add block orders

# Block supply offers: (name, zone, MW, price, start_hour, end_hour)
block_supply = [
    ("NGA Baseload Night", 'NGA', 500, 35, 0, 7),     # cheap night block
    ("GHA Peak Block",     'GHA', 300, 65, 17, 22),    # evening peaker
    ("CIV Solar Block",    'CIV', 200, 30, 8, 16),     # daytime solar
    ("NGA Evening Gas",    'NGA', 400, 42, 16, 23),    # evening gas
]

# Block demand bids: (name, zone, MW, max_price, start_hour, end_hour)
block_demand = [
    ("Industrial NGA",  'NGA', 600, 100, 8, 17),       # daytime industrial
    ("BFA Evening",     'BFA', 200, 180, 18, 23),      # evening consumption
    ("SEN Night Base",  'SEN', 150, 140, 0, 5),        # night base load
]

BS = range(len(block_supply)); BD_b = range(len(block_demand))

# --- Model with blocks ---
m2 = ConcreteModel()

# Hourly variables (same as before)
m2.xs = Var(S, T, bounds=(0,1))
m2.xd = Var(D, T, bounds=(0,1))
m2.f  = Var(PAIRS, T, domain=NonNegativeReals)
m2.fr = Var(PAIRS, T, domain=NonNegativeReals)
m2.b  = Var(PAIRS, T, domain=Binary)

# Block variables
m2.ybs = Var(BS, domain=Binary)      # block supply acceptance
m2.ybd = Var(BD_b, domain=Binary)    # block demand acceptance

# Objective: hourly welfare + block welfare
hourly_welfare = (
    sum(demand[d][3]*Qd[d][t]*m2.xd[d,t] for t in T for d in D if Qd[d][t]>0)
  - sum(supply[s][3]*Qs[s][t]*m2.xs[s,t] for t in T for s in S if Qs[s][t]>0))

block_welfare = (
    sum(block_demand[bd][3]*block_demand[bd][2]*(block_demand[bd][5]-block_demand[bd][4]+1)*m2.ybd[bd] for bd in BD_b)
  - sum(block_supply[bs][3]*block_supply[bs][2]*(block_supply[bs][5]-block_supply[bs][4]+1)*m2.ybs[bs] for bs in BS)
)

m2.obj = Objective(expr=hourly_welfare + block_welfare, sense=maximize)
m2.dual = Suffix(direction=Suffix.IMPORT)

# Nodal balance including blocks
def balance2(m2, z, t):
    prod = sum(Qs[s][t]*m2.xs[s,t] for s in S if supply[s][1]==z and Qs[s][t]>0)
    cons = sum(Qd[d][t]*m2.xd[d,t] for d in D if demand[d][1]==z and Qd[d][t]>0)

    # Block supply injections
    prod += sum(block_supply[bs][2]*m2.ybs[bs] for bs in BS
                if block_supply[bs][1]==z and block_supply[bs][4] <= t <= block_supply[bs][5])
    # Block demand withdrawals
    cons += sum(block_demand[bd][2]*m2.ybd[bd] for bd in BD_b
                if block_demand[bd][1]==z and block_demand[bd][4] <= t <= block_demand[bd][5])

    imp = sum(m2.f[u,v,t] for (u,v) in PAIRS if v==z) + sum(m2.fr[u,v,t] for (u,v) in PAIRS if u==z)
    exp = sum(m2.f[u,v,t] for (u,v) in PAIRS if u==z) + sum(m2.fr[u,v,t] for (u,v) in PAIRS if v==z)
    return prod + imp - exp == cons

m2.bal = Constraint(ZONES, T, rule=balance2)
m2.ntc_f = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.f[u,v,t] <= NTC[(u,v)]*m.b[u,v,t])
m2.ntc_r = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.fr[u,v,t] <= NTC[(u,v)]*(1-m.b[u,v,t]))

# Solve
t0 = time.time()
SolverFactory('glpk').solve(m2, tee=True)
for (u,v) in PAIRS:
    for t in T:
        m2.b[u,v,t].fix(round(value(m2.b[u,v,t])))
SolverFactory('glpk').solve(m2, tee=True)
dt = time.time() - t0

# Extract prices
prices_zt2 = {}
for z in ZONES:
    prices_zt2[z] = []
    for t in T:
        p = -m2.dual.get(m2.bal[z,t], 0)
        if p < 0:
            acc = [(supply[s][3], value(m2.xs[s,t])) for s in S
                   if supply[s][1]==z and Qs[s][t]>0
                   and value(m2.xs[s,t]) is not None and value(m2.xs[s,t])>0.01]
            p = max([pr for pr,_ in acc], default=0)
        prices_zt2[z].append(round(p, 2))

print(f"Solved in {dt:.1f}s")
print(f"Total welfare (with blocks): {value(m2.obj):,.0f} €\n")

# Block results
print("Block supply orders:")
for bs in BS:
    name, zone, mw, price, h0, h1 = block_supply[bs]
    accepted = value(m2.ybs[bs]) > 0.5
    avg_price = np.mean([prices_zt2[zone][t] for t in range(h0, h1+1)])
    in_money = avg_price >= price
    status = "✓ ACCEPTED" if accepted else "✗ REJECTED"
    prb = " (PRB!)" if in_money and not accepted else ""
    pab = " (PAB!)" if not in_money and accepted else ""
    print(f"  {status} {name:25s} {mw:4d} MW @ {price:3d} € H{h0:02d}-H{h1:02d}  avg_λ={avg_price:.1f}{prb}{pab}")

print("\nBlock demand orders:")
for bd in BD_b:
    name, zone, mw, price, h0, h1 = block_demand[bd]
    accepted = value(m2.ybd[bd]) > 0.5
    avg_price = np.mean([prices_zt2[zone][t] for t in range(h0, h1+1)])
    in_money = avg_price <= price
    status = "✓ ACCEPTED" if accepted else "✗ REJECTED"
    prb = " (PRB!)" if in_money and not accepted else ""
    pab = " (PAB!)" if not in_money and accepted else ""
    print(f"  {status} {name:25s} {mw:4d} MW @ {price:3d} € H{h0:02d}-H{h1:02d}  avg_λ={avg_price:.1f}{prb}{pab}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\aymar\AppData\Local\Temp\tmp5537ecci.glpk.raw --wglp C:\Users\aymar\AppData\Local\Temp\tmpkyq56q3y.glpk.glp
 --cpxlp C:\Users\aymar\AppData\Local\Temp\tmpqzcxj9od.pyomo.lp
Reading problem data from 'C:\Users\aymar\AppData\Local\Temp\tmpqzcxj9od.pyomo.lp'...
C:\Users\aymar\AppData\Local\Temp\tmpqzcxj9od.pyomo.lp:10271: warning: lower bound of variable 'x1024' redefined
C:\Users\aymar\AppData\Local\Temp\tmpqzcxj9od.pyomo.lp:10271: warning: upper bound of variable 'x1024' redefined
1056 rows, 2109 columns, 3955 non-zeros
367 integer variables, all of which are binary
10638 lines were read
Writing problem data to 'C:\Users\aymar\AppData\Local\Temp\tmpkyq56q3y.glpk.glp'...
10614 lines were written
GLPK Integer Optimizer 5.0
1056 rows, 2109 columns, 3955 non-zeros
367 integer variables, all of which are binary
Preprocessing...
3 constraint coefficient(s) were reduced
1059 rows, 1773 columns, 3637 non

In [16]:
# Visualize block order impact on prices
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.08,
    subplot_titles=["Prices with Block Orders (NGA)", "Block Order Schedule"])

# NGA prices comparison
fig.add_trace(go.Scatter(x=T, y=prices_zt['NGA'], mode='lines', name='Without blocks',
    line=dict(color='gray', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=T, y=prices_zt2['NGA'], mode='lines', name='With blocks',
    line=dict(color='#2B4C7E', width=2)), row=1, col=1)

# Block schedule (Gantt-like)
colors_block = {'supply': 'rgba(39,174,96,0.7)', 'demand': 'rgba(231,76,60,0.7)'}
for i, (name, zone, mw, price, h0, h1) in enumerate(block_supply):
    fig.add_trace(go.Bar(x=[h1-h0+1], y=[name], base=[h0], orientation='h',
        marker_color=colors_block['supply'], showlegend=i==0,
        name='Supply block', width=0.6,
        text=f"{mw}MW@{price}€ {'✓' if value(m2.ybs[i])>0.5 else '✗'}",
        textposition='inside'), row=2, col=1)

for i, (name, zone, mw, price, h0, h1) in enumerate(block_demand):
    fig.add_trace(go.Bar(x=[h1-h0+1], y=[name], base=[h0], orientation='h',
        marker_color=colors_block['demand'], showlegend=i==0,
        name='Demand block', width=0.6,
        text=f"{mw}MW@{price}€ {'✓' if value(m2.ybd[i])>0.5 else '✗'}",
        textposition='inside'), row=2, col=1)

fig.update_layout(height=550, barmode='overlay')
fig.update_xaxes(title_text="Hour", dtick=2, row=2, col=1)
fig.update_yaxes(title_text="€/MWh", row=1, col=1)
fig.show()

---
## Step 5 — Complex Orders

### 5a. Linked Orders

A **linked order** $l$ can only be accepted if its parent order $\pi(l)$ is also accepted:

$$y_l \leq y_{\pi(l)} \quad \forall l \in \mathcal{L}_{\text{linked}}$$

Use case: a gas plant (parent) and its must-run auxiliary (child).

### 5b. Exclusive Groups

Orders in an exclusive group $\mathcal{G}$ are mutually exclusive — at most one can be accepted:

$$\sum_{b \in \mathcal{G}} y_b \leq 1$$

Use case: a plant offers different operating modes (low/medium/high output) as separate blocks.

### 5c. Minimum Income Condition (MIC)

A generator requires that its total revenue minus variable costs exceeds a fixed cost $F_s$:

$$\sum_{t \in \mathcal{T}} \lambda_{z,t} \cdot \bar{q}_{s,t} \cdot x^s_{s,t} - \sum_{t} p_s \cdot \bar{q}_{s,t} \cdot x^s_{s,t} \geq F_s \cdot y^{\text{mic}}_s$$

Since $\lambda_{z,t}$ is a dual (not known ex ante), this is typically handled via **iterative rejection**: solve, check MIC, reject violators, re-solve.

**Note**: MIC makes the problem non-convex and requires iterative heuristics similar to EUPHEMIA's approach.

In [17]:
# --- 5a & 5b: Linked orders + Exclusive groups ---

# Parent block: baseload gas plant
# Child block: auxiliary (must-run if parent runs)
# Exclusive group: same plant, 3 output levels

block_supply_cx = [
    # (name, zone, MW, price, h0, h1, parent_idx, excl_group)
    ("NGA Gas Full",    'NGA', 800, 33, 6, 22, None,  'NGA_gas'),   # idx 0
    ("NGA Gas Medium",  'NGA', 500, 35, 6, 22, None,  'NGA_gas'),   # idx 1
    ("NGA Gas Low",     'NGA', 250, 38, 6, 22, None,  'NGA_gas'),   # idx 2
    ("NGA Gas Aux",     'NGA', 50,  25, 6, 22, 0,     None),        # idx 3, child of 0
    ("GHA Peak Block",  'GHA', 300, 65, 17, 22, None,  None),       # idx 4
    ("CIV Solar Block", 'CIV', 200, 30, 8, 16, None,  None),       # idx 5
]

block_demand_cx = [
    ("Industrial NGA",  'NGA', 600, 100, 8, 17, None, None),
    ("BFA Evening",     'BFA', 200, 180, 18, 23, None, None),
]

BS_cx = range(len(block_supply_cx)); BD_cx = range(len(block_demand_cx))

# --- Model ---
m3 = ConcreteModel()
m3.xs = Var(S, T, bounds=(0,1))
m3.xd = Var(D, T, bounds=(0,1))
m3.f  = Var(PAIRS, T, domain=NonNegativeReals)
m3.fr = Var(PAIRS, T, domain=NonNegativeReals)
m3.b  = Var(PAIRS, T, domain=Binary)
m3.ybs = Var(BS_cx, domain=Binary)
m3.ybd = Var(BD_cx, domain=Binary)

# Objective
hourly_w = (
    sum(demand[d][3]*Qd[d][t]*m3.xd[d,t] for t in T for d in D if Qd[d][t]>0)
  - sum(supply[s][3]*Qs[s][t]*m3.xs[s,t] for t in T for s in S if Qs[s][t]>0))

block_w = (
    sum(block_demand_cx[bd][3]*block_demand_cx[bd][2]*(block_demand_cx[bd][5]-block_demand_cx[bd][4]+1)*m3.ybd[bd] for bd in BD_cx)
  - sum(block_supply_cx[bs][3]*block_supply_cx[bs][2]*(block_supply_cx[bs][5]-block_supply_cx[bs][4]+1)*m3.ybs[bs] for bs in BS_cx)
)
m3.obj = Objective(expr=hourly_w + block_w, sense=maximize)
m3.dual = Suffix(direction=Suffix.IMPORT)

# Balance
def balance3(m3, z, t):
    prod = sum(Qs[s][t]*m3.xs[s,t] for s in S if supply[s][1]==z and Qs[s][t]>0)
    cons = sum(Qd[d][t]*m3.xd[d,t] for d in D if demand[d][1]==z and Qd[d][t]>0)
    prod += sum(block_supply_cx[bs][2]*m3.ybs[bs] for bs in BS_cx
                if block_supply_cx[bs][1]==z and block_supply_cx[bs][4]<=t<=block_supply_cx[bs][5])
    cons += sum(block_demand_cx[bd][2]*m3.ybd[bd] for bd in BD_cx
                if block_demand_cx[bd][1]==z and block_demand_cx[bd][4]<=t<=block_demand_cx[bd][5])
    imp = sum(m3.f[u,v,t] for (u,v) in PAIRS if v==z) + sum(m3.fr[u,v,t] for (u,v) in PAIRS if u==z)
    exp = sum(m3.f[u,v,t] for (u,v) in PAIRS if u==z) + sum(m3.fr[u,v,t] for (u,v) in PAIRS if v==z)
    return prod + imp - exp == cons
m3.bal = Constraint(ZONES, T, rule=balance3)
m3.ntc_f = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.f[u,v,t]<=NTC[(u,v)]*m.b[u,v,t])
m3.ntc_r = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.fr[u,v,t]<=NTC[(u,v)]*(1-m.b[u,v,t]))

# --- LINKED ORDER constraint: child <= parent ---
m3.linked = ConstraintList()
for bs in BS_cx:
    parent = block_supply_cx[bs][6]
    if parent is not None:
        m3.linked.add(m3.ybs[bs] <= m3.ybs[parent])

# --- EXCLUSIVE GROUP constraint: at most one in group ---
excl_groups = {}
for bs in BS_cx:
    grp = block_supply_cx[bs][7]
    if grp is not None:
        excl_groups.setdefault(grp, []).append(bs)

m3.excl = ConstraintList()
for grp, members in excl_groups.items():
    m3.excl.add(sum(m3.ybs[bs] for bs in members) <= 1)

print("Exclusive groups:")
for grp, members in excl_groups.items():
    print(f"  {grp}: {[block_supply_cx[bs][0] for bs in members]}")
print(f"\nLinked orders:")
for bs in BS_cx:
    parent = block_supply_cx[bs][6]
    if parent is not None:
        print(f"  {block_supply_cx[bs][0]} → requires {block_supply_cx[parent][0]}")

# Solve
t0 = time.time()
SolverFactory('glpk').solve(m3, tee=False)
for (u,v) in PAIRS:
    for t in T:
        m3.b[u,v,t].fix(round(value(m3.b[u,v,t])))
SolverFactory('glpk').solve(m3, tee=False)
dt = time.time()-t0

print(f"\nSolved in {dt:.1f}s")
print(f"Total welfare: {value(m3.obj):,.0f} €\n")

print("Block results:")
for bs in BS_cx:
    name, zone, mw, price, h0, h1, parent, excl = block_supply_cx[bs]
    acc = value(m3.ybs[bs]) > 0.5
    tag = "✓" if acc else "✗"
    extra = ""
    if excl: extra += f" [group:{excl}]"
    if parent is not None: extra += f" [child of {block_supply_cx[parent][0]}]"
    print(f"  {tag} {name:20s} {mw:4d} MW @ {price:3d} €  H{h0:02d}-H{h1:02d}{extra}")

Exclusive groups:
  NGA_gas: ['NGA Gas Full', 'NGA Gas Medium', 'NGA Gas Low']

Linked orders:
  NGA Gas Aux → requires NGA Gas Full

Solved in 1.8s
Total welfare: 18,849,321 €

Block results:
  ✓ NGA Gas Full          800 MW @  33 €  H06-H22 [group:NGA_gas]
  ✗ NGA Gas Medium        500 MW @  35 €  H06-H22 [group:NGA_gas]
  ✗ NGA Gas Low           250 MW @  38 €  H06-H22 [group:NGA_gas]
  ✓ NGA Gas Aux            50 MW @  25 €  H06-H22 [child of NGA Gas Full]
  ✓ GHA Peak Block        300 MW @  65 €  H17-H22
  ✓ CIV Solar Block       200 MW @  30 €  H08-H16


### 5c — Minimum Income Condition (MIC) via Iterative Rejection

Since MIC depends on clearing prices (dual variables), it cannot be directly included in the LP/MIP. We use an **iterative approach**:

1. Solve the market clearing
2. For each generator with a MIC, check if revenue ≥ fixed cost
3. If not, **reject** the generator (fix $x^s_{s,t} = 0$) and re-solve
4. Repeat until no MIC violations remain

This is a simplified version of the EUPHEMIA PRB/PAB elimination loop.

In [18]:
# MIC: some generators have fixed costs that must be covered
# (generator_index, fixed_cost_EUR)
mic_generators = [
    (5,  8000),    # Olorunsogo (peaker): 8000€ startup cost
    (9,  12000),   # Karpowership (GHA): 12000€ daily fixed cost
    (13, 5000),    # Aggreko CIV: 5000€ daily cost
]

# Start from Step 3 model (rebuild fresh)
def solve_with_mic(mic_gens, max_iter=10):
    rejected = set()

    for iteration in range(max_iter):
        m_mic = ConcreteModel()
        active_S = [s for s in S if s not in rejected]

        m_mic.xs = Var(S, T, bounds=(0,1))
        m_mic.xd = Var(D, T, bounds=(0,1))
        m_mic.f  = Var(PAIRS, T, domain=NonNegativeReals)
        m_mic.fr = Var(PAIRS, T, domain=NonNegativeReals)
        m_mic.b  = Var(PAIRS, T, domain=Binary)

        # Fix rejected generators to 0
        for s_rej in rejected:
            for t in T:
                m_mic.xs[s_rej, t].fix(0)

        m_mic.obj = Objective(expr=
            sum(demand[d][3]*Qd[d][t]*m_mic.xd[d,t] for t in T for d in D if Qd[d][t]>0)
          - sum(supply[s][3]*Qs[s][t]*m_mic.xs[s,t] for t in T for s in active_S if Qs[s][t]>0),
            sense=maximize)

        m_mic.dual = Suffix(direction=Suffix.IMPORT)

        def bal(m_mic, z, t):
            prod = sum(Qs[s][t]*m_mic.xs[s,t] for s in S if supply[s][1]==z and Qs[s][t]>0)
            cons = sum(Qd[d][t]*m_mic.xd[d,t] for d in D if demand[d][1]==z and Qd[d][t]>0)
            imp = sum(m_mic.f[u,v,t] for (u,v) in PAIRS if v==z) + sum(m_mic.fr[u,v,t] for (u,v) in PAIRS if u==z)
            exp = sum(m_mic.f[u,v,t] for (u,v) in PAIRS if u==z) + sum(m_mic.fr[u,v,t] for (u,v) in PAIRS if v==z)
            return prod + imp - exp == cons
        m_mic.bal = Constraint(ZONES, T, rule=bal)
        m_mic.ntc_f = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.f[u,v,t]<=NTC[(u,v)]*m.b[u,v,t])
        m_mic.ntc_r = Constraint(PAIRS, T, rule=lambda m,u,v,t: m.fr[u,v,t]<=NTC[(u,v)]*(1-m.b[u,v,t]))

        SolverFactory('glpk').solve(m_mic, tee=False)
        for (u,v) in PAIRS:
            for t in T:
                m_mic.b[u,v,t].fix(round(value(m_mic.b[u,v,t])))
        SolverFactory('glpk').solve(m_mic, tee=False)

        # Extract prices
        prices_mic = {}
        for z in ZONES:
            prices_mic[z] = []
            for t in T:
                p = -m_mic.dual.get(m_mic.bal[z,t], 0)
                if p < 0:
                    acc = [(supply[s][3], value(m_mic.xs[s,t])) for s in active_S
                           if supply[s][1]==z and Qs[s][t]>0
                           and value(m_mic.xs[s,t]) is not None and value(m_mic.xs[s,t])>0.01]
                    p = max([pr for pr,_ in acc], default=0)
                prices_mic[z].append(round(p, 2))

        # Check MIC
        violations = []
        for s_idx, fixed_cost in mic_gens:
            if s_idx in rejected:
                continue
            zone = supply[s_idx][1]
            var_cost = supply[s_idx][3]

            revenue = sum(prices_mic[zone][t] * Qs[s_idx][t] * value(m_mic.xs[s_idx,t]) for t in T if Qs[s_idx][t]>0)
            cost = sum(var_cost * Qs[s_idx][t] * value(m_mic.xs[s_idx,t]) for t in T if Qs[s_idx][t]>0)
            margin = revenue - cost
            energy = sum(Qs[s_idx][t] * value(m_mic.xs[s_idx,t]) for t in T if Qs[s_idx][t]>0)

            if energy > 0 and margin < fixed_cost:
                violations.append((s_idx, margin, fixed_cost))

        print(f"Iteration {iteration+1}: welfare={value(m_mic.obj):,.0f}€, "
              f"rejected={[supply[s][0] for s in rejected]}, "
              f"MIC violations={len(violations)}")

        for s_idx, margin, fc in violations:
            print(f"  → Reject {supply[s_idx][0]}: margin={margin:,.0f}€ < fixed_cost={fc:,.0f}€")

        if not violations:
            print(f"\nConverged after {iteration+1} iterations")
            return m_mic, prices_mic, rejected

        # Reject the generator with worst violation
        worst = min(violations, key=lambda x: x[1] - x[2])
        rejected.add(worst[0])

    return m_mic, prices_mic, rejected

m_final, prices_final, rejected = solve_with_mic(mic_generators)
print(f"\nFinal welfare: {value(m_final.obj):,.0f} €")
print(f"Rejected generators: {[supply[s][0] for s in rejected]}")

Iteration 1: welfare=17,450,604€, rejected=[], MIC violations=1
  → Reject Karpowership: margin=0€ < fixed_cost=12,000€
Iteration 2: welfare=16,987,536€, rejected=['Karpowership'], MIC violations=0

Converged after 2 iterations

Final welfare: 16,987,536 €
Rejected generators: ['Karpowership']


---
## Summary

| Step | Variables | Constraints | Key result |
|------|-----------|-------------|------------|
| 1. Single zone | $x^s, x^d$ continuous | 1 balance | Single clearing price $\lambda$ |
| 2. Multi-zone + NTC | + flows $f$, direction $b$ binary | 14 balance + 30 NTC | Zonal prices $\lambda_z$, congestion rents |
| 3. Multi-period 24h | All indexed by $t$ | 14×24 balance + 30×24 NTC | Price curves $\lambda_{z,t}$, dispatch profiles |
| 4. Block orders | + $y_b$ binary | Balance includes blocks | Fill-or-kill, PRB/PAB detection |
| 5. Complex orders | + linked/exclusive constraints | $y_{child} \leq y_{parent}$, $\sum y \leq 1$ | Realistic market design |

### Key references
- Madani & Van Vyve (2017) — *Revisiting minimum profit conditions in uniform price day-ahead electricity auctions*
- EUPHEMIA algorithm documentation (PCR/NEMO Committee)
- ERERA Resolution 010/2017 — *Electricity Market Procedures for WAPP*